# 3. 自己位置推定と自律移動

<div class="alert alert-block alert-info">
    <b>この章の目的</b>
    <p>ロボットを指定した場所まで障害物を避けながら自律移動させる方法を学習します</p>
</div> 

# RVizを使った自律移動

rvizを起動します。下記コマンドを実行して、起動してください。

In [ ]:
%%script bash --bg
rviz -d data/3_navigation.rviz > /dev/null 2>&1

シミュレータ画面で、rvizが立ち上がったことを確認してください。rvizでは、ロボットに搭載された各種センサ情報を確認できます。

![title](./imgs/3_rviz.png)

「Displays」の中のチェックをON, OFFして表示されている情報を確認してみましょう。それぞれの情報の意味は以下の通りです。

- Map: 事前に与えられた地図情報


- global_cost_map: 事前地図を元に生成したコストマップ


- local_cost_map: レーザ情報を元に生成したコストマップ

ここでコストマップとは、通れそうな領域、障害物のある領域を数値で表現した地図のことで、自律移動の際に用いられます。

rvizから自律移動のゴールを入力します。rvizの上部のツールバーにある「2D Nav Goal」をクリックし、部屋内の障害物のない場所を指定してみましょう。

![title](./imgs/3_2d_nav_goal_button.png)
![title](./imgs/3_2d_nav_goal.png)


HSRが自律移動を開始します。rviz画面や、シミュレータ画面で、HSRが障害物を避けながら指定した位置に移動することを確認しましょう。
- rviz画面
![title](./imgs/3_moving.png)


- gazebo画面
![title](./imgs/3_moving_gazebo.png)

## 障害物回避
次に、人の位置を移動させてみましょう。gazeboの上にある
![title](./imgs/3_move_object_button.png)
このボタンをクリックしてから、人をクリックしてください。
![title](./imgs/3_human_select.png)
青、赤、緑の矢印が表示されました。赤と緑の矢印を動かすと、人をXY平面上にて動かせます。人を動かしている時に、人をロボットにぶつけてまうと、ロボットの挙動がおかしくなるので、ぶつからないように移動させましょう。

終わったら、矢印ボタンを押して、選択状態を終了しましょう。
![title](./imgs/3_back_to_default.png)

rvizの画面に戻って、「2D Nav Goal」を与え、HSRを自律移動させましょう。

![title](./imgs/3_move_avoiding_human.png)

人をよけながら自律移動できたでしょうか？

# Pythonスクリプトからの自律移動

前章はrvizから自律移動を実行しましたが、今回はPythonから指定した座標に自律移動する指令を出してみましょう。

### 1. 準備
まず、必要なライブラリをインポートし、初期化します。

In [ ]:
import rospy
from utils import *
rospy.init_node('navigation')

### 2. ゴール座標指定
次に、ゴールの座標を決めましょう。rvizの右上の「Publish Point」ボタンをクリックします。
![title](./imgs/3_publish_point_button.png)


次に、地図上で目的地にマウスを持って行くと、rvizの左下の「Select this point」の部分に座標が表示されます。この例では、x=1.29,y=3.13,z=0.00747 が目的地の座標です。zは不要なので、x,yの値をメモしておきましょう。
![title](./imgs/3_point_values.png)

#### 注意: ゴールを地図から外れた位置に設定すると誤動作するので、地図内の位置にしてください。

### 3. ゴールを送る
ロボットにゴールを送りましょう。下記のコマンドを実行し、シミュレータ上でのロボットの動作を観察してください。

台車を自律移動させるには`move_base_goal`という関数を使用します。興味のある人は見てみてください。

```python
# 自律移動のゴールを送信するクライアントを作成
navclient = actionlib.SimpleActionClient('/move_base', MoveBaseAction)

def move_base_goal(x, y, theta):
    u"""台車の自律移動のゴールを指定する関数

    引数：
        x (float): 目標のx値 [m]
        y (float): 目標のy値 [m]
        theta (float): 目標の回転角度 [deg]

    返り値:
        ゴールに到達したらTrue, そうでなければFalse

    """

    goal = MoveBaseGoal()

    # "map"座標を基準座標に指定
    goal.target_pose.header.frame_id = "map"

    # ゴールのx,y座標をセットします
    goal.target_pose.pose.position.x = x
    goal.target_pose.pose.position.y = y

    # 角度はクオータニオンという形式で与えます。そのため、オイラー角からクオータニオンに変換します
    goal.target_pose.pose.orientation = quaternion_from_euler(0, 0, theta)

    # ゴールを送信
    navclient.send_goal(goal)
    navclient.wait_for_result()
    state = navclient.get_state()
    # 成功すると、3が返ってくる
    # http://docs.ros.org/fuerte/api/actionlib_msgs/html/msg/GoalStatus.html
    return True if state == 3 else False
```

以下のコマンドでx=1メートル、y=0.5メートル、theta=90度の位置姿勢に移動できます。

In [ ]:
move_base_goal(1, 0.5, 90)

以下のコマンドで原点に戻ります。

In [ ]:
move_base_goal(0, 0, 0)

<div class="alert alert-block alert-info">
    <b>課題</b>
    <p>ある場所についたら次のゴールに自動的に向かうようにして、部屋を周回するプログラムを書いてみましょう。</p>
</div>

In [ ]:
# 自分で考えてみましょう。この下に入力できます。


<div class="alert alert-block alert-info">
    <b>アドバンスド課題</b>
    <p>障害物の上にゴールを置くとどうなるでしょうか？</p>
    <p>自律移動中に人を動かして、ロボットの邪魔をするとどうなるでしょうか？</p>
    <p>地図の外にゴールを置くとどうなるでしょうか</p>
</div>

In [ ]:
# 自分で考えてみましょう。この下に入力できます。


<div class="alert alert-block alert-info">
    <b>次の学習</b>
    <p>今までの章では台車の制御を扱ってきました。次の章ではHSRのアームを制御してみましょう。</p>
</div>